<a href="https://colab.research.google.com/github/felixzhao/title_catgories_classification/blob/main/JTC_v3_1_level2_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install dependency

In [1]:
! pip install sentence-transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 20.7 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=aafb4f5df329b04115edc130e58b365fd5515875e303bad7f45ec63c38090aca
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers


# load packages

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

from collections import defaultdict
import json

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,classification_report

from sentence_transformers import SentenceTransformer, util
from sklearn import metrics

load data
This work loads data from google drive. Please put the files into "trademe_data" folder under the root of google drive.

Then run the following code to mount the google drive in colab.

You can follow the guide. https://towardsdatascience.com/different-ways-to-connect-google-drive-to-a-google-colab-notebook-pt-1-de03433d2f7a

In [4]:
from google.colab import drive
drive.mount('/content/drive')
root_path = "drive/MyDrive/trademe_data/"

Mounted at /content/drive


# load  data

In [38]:
raw_path = f"{root_path}raw.csv"

In [39]:
raw_df = pd.read_csv(raw_path)

In [40]:
raw_df = raw_df[['x_title_feature', 'cat_2']]
print(len(raw_df))
raw_df.head()

20000


,x_title_feature,cat_2
0,unqualifi asbesto remov,labourers
1,senior test analyst,testing
2,ict trainer supervisor,tutoring & training
3,automot specialist multi facet posit,technicians
4,busi analyst,business & systems analysts


# preprocess

In [41]:
dups = raw_df[raw_df.duplicated()]
print(len(dups))
dups[:1]

5113


,x_title_feature,cat_2
72,support worker,community & social services


In [42]:
raw_df = raw_df.drop_duplicates()
print(len(raw_df))

14887


# encoding

In [43]:
model = SentenceTransformer("all-MiniLM-L6-v2")

In [44]:

# def get_embedding(text):
#     if not text:
#         print("Content is empty.")
#         return None
#     return model.encode(input, convert_to_tensor=True)
     

In [45]:
# def get_nparray_embdding(content):
#     return content.cpu().numpy()

In [67]:
raw_df['x_title_feature_encoding'] = raw_df['x_title_feature'].apply(lambda x: model.encode(str(x), convert_to_tensor=True).cpu().numpy())

In [64]:
raw_df.head(2)

,x_title_feature,cat_2,x_title_feature_encoding
0,unqualifi asbesto remov,labourers,"[-0.04014635, 0.04006017, -0.052789543, 0.0232..."
1,senior test analyst,testing,"[-0.02413135, 0.055597737, -0.103181325, 0.024..."


# Train & test split

In [80]:
X_train, X_test , y_train, y_test = train_test_split(raw_df['x_title_feature_encoding'],   raw_df[['cat_2']],  test_size=0.2,   random_state=123    )

In [81]:
X_train = X_train.to_list()
X_test = X_test.to_list()

# Classification

In [82]:
from typing import Any, Optional

import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import (
    classification_report,
    accuracy_score,
    confusion_matrix,
    ConfusionMatrixDisplay
)
import pickle

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

In [83]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)

<ipython-input-83-549420f83866>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train, y_train)


RandomForestClassifier()

In [84]:
preds = clf.predict(X_test)

In [92]:
from datetime import datetime
start = datetime.utcnow()

t = clf.predict([X_test[0]])

date= datetime.utcnow() - start
print("Number of days since epoch:",date)
seconds =(date.total_seconds())
milliseconds = round(seconds*1000)
print("Milliseconds since epoch:",milliseconds)

Number of days since epoch: 0:00:00.017530
Milliseconds since epoch: 18


In [93]:
probas = clf.predict_proba(X_test)

In [94]:
report = classification_report(y_test, preds)
print(report)

                                             precision    recall  f1-score   support

                         account management       0.00      0.00      0.00         8
                                accountants       0.46      0.46      0.46        39
                    accounts administrators       0.00      0.00      0.00         9
                           accounts payable       0.22      0.25      0.24         8
                        accounts receivable       0.00      0.00      0.00         4
                             administration       0.35      0.67      0.46       135
                                advertising       0.00      0.00      0.00         3
                   air con. & refrigeration       0.67      0.57      0.62        21
                                   analysts       1.00      0.14      0.25         7
                                 architects       0.45      0.56      0.50         9
                   assistant store managers       0.50      0.31

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1